In [1]:
import openai
import pandas as pd
import time
import os
import base64

In [2]:
# from transformers import BitsAndBytesConfig
# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,  # Use True for 4-bit, False for 8-bit
#     bnb_4bit_compute_dtype="float16",  # Recommended dtype
#     bnb_4bit_use_double_quant=True,  # Enables double quantization
#     bnb_4bit_quant_type="nf4"  # Normalized float4 (better for LLMs)
# )

In [3]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline
# messages = [
#     {"role": "user", "content": "Who are you?"},
# ]
# pipe = pipeline("text-generation", model="unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit")
# pipe(messages)

### Llama 3.2 11B Vision model in 4bit config generates around 1024 tokens per min with RTX4070 (16GB)
### how to speed up the inference with limited GPU

In [4]:
# # Load model directly
# from transformers import AutoProcessor, AutoModelForImageTextToText

# processor = AutoProcessor.from_pretrained("unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit")
# model = AutoModelForImageTextToText.from_pretrained("unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit")


# # Define the text input (prompt)
# instruction = "Write a creative story about an astronaut exploring a new planet."

# # Process the text input (no image required)
# inputs = processor(text=instruction, return_tensors="pt")

# # Ensure that input_ids are on the same device as the model
# device = model.device
# inputs = inputs.to(device)  # Move inputs to the correct device

# start_time = time.time()
# # Perform inference using the model
# outputs = model.generate(**inputs,max_new_tokens=1024)
# end_time = time.time()

# # Calculate the time taken
# elapsed_time = end_time - start_time
# print(f"Time taken to generate 1024 tokens: {elapsed_time:.2f} seconds")

# # Decode and print the output
# decoded_output = processor.decode(outputs[0], skip_special_tokens=True)
# print(decoded_output)

In [5]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

modelV, tokenizerV = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Mllama vision patching. Transformers: 4.46.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti SUPER. Max memory: 15.693 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Llama 3.2 11B Vision model (unsloth) in 4bit config generates around 1500 tokens per min with RTX4070 (16GB)
### ### Image + text to text

In [6]:
from PIL import Image

FastVisionModel.for_inference(modelV) # Enable for inference!

image_path = "ex1.jpg"  # Change this to your image path
image = Image.open(image_path).convert("RGB")

instruction = "Describe accurately what you see in this image."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizerV.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizerV(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizerV, skip_prompt = True)

_ = modelV.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The image presents a diagram of a rotating mechanical device that illustrates how an object's rotation creates a torque that induces forces to the end of the object, commonly found in rotational dynamics.

A slender object, labelled O, is drawn in blue, and a white cylinder appears to be grasped by its end, creating a force, denoted by the letter F. The diagram labels two additional letters: the variable z and the letter d. The line labelled 'z' indicates an imaginary line that appears to originate from the base of the object that passes through its point of contact, which is a circle representing an end-cap that is perpendicular to the


### Llama 3.2 3B text only model (unsloth) in 4bit config generates around ?? tokens per min with RTX4070 (16GB)
### ### Text to text

In [7]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models
fourbit_models = [
    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] 


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.46.1.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti SUPER. Max memory: 15.693 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [8]:
### Chatml style format
### Uses OpenAI’s chatml format, which structures messages in a way that aligns with popular models like Zephyr, Mistral, LLaMA, etc.
### Converts roles (user, assistant) into a standardized format (human, gpt) to match the expected input for the model.
### Ensures that the model properly recognizes the end-of-sentence token (</s>), preventing endless text generation.

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "write a story about an astronaut"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True)
tokenizer.batch_decode(outputs)

Unsloth: Will map <|im_end|> to EOS = <|eot_id|>.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|im_start|>user\nwrite a story about an astronaut<|im_end|>\n<|im_start|>assistant\nAs the spaceship soared through the vast expanse of space, astronaut Jack Harris gazed out at the stars, feeling a sense of awe and wonder that never grew old. He had spent years training for this mission, and now, finally, he was on his way to explore the distant reaches of the galaxy.\n\nJack\'s ship, the Aurora, was equipped with state-of-the-art technology and a crew of seasoned astronauts. Together, they had set out on a journey to discover new worlds, to push the boundaries of human knowledge, and to seek out new life and civilizations.\n\nAs the days passed, the Aurora pierced the veil of the cosmos, entering the unknown territories of the galaxy. Jack marveled at the breathtaking vistas, the swirling nebulae, and the majestic galaxies that stretched out before him like a canvas of celestial beauty.\n\nBut Jack\'s journey was not without its challenges. The ship was equipped with a sophisticat

In [9]:
### text streamer format
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "write a story about an astronaut"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

<|im_start|>user
write a story about an astronaut<|im_end|>
<|im_start|>assistant
As the stars whizzed by like diamonds on velvet, astronaut Jack Harris gazed out the window of the International Space Station. The curvature of the Earth, a breathtaking sight, filled his view. He felt tiny, yet connected to the vastness of space.

Jack's thoughts drifted back to his childhood, growing up in a small town in the American Midwest. He always felt like an outsider, like he didn't quite fit in. But when he saw the first moon landing on TV, something clicked. He knew he wanted to explore the unknown, to push the boundaries of human knowledge.

Years of hard work, dedication, and perseverance had led him to this moment. Jack was about to embark on his first spacewalk, a milestone in his career as an astronaut. His heart pounded with excitement and a hint of nervousness.

"Ready for the spacewalk, Jack?" his commander, Captain Lewis, asked over the comms system.

"Roger that, Captain," Jack repl